# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import LSTM_AutoEncoder as Model
from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/LSTM_AE_apecaug/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters())
    criterion = nn.MSELoss()
    num_epochs = config['fit']['num_epochs']
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    #if (machine_type == machine_types[0]) or (machine_type == machine_types[1]):
    #    continue
    history = run(machine_type)
    with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
        pickle.dump(history , file)

2020-11-01 00:43:33,155 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-01 00:43:33,156 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-01 00:43:34,324 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 1803/1803 [00:26<00:00, 66.93it/s]
2020-11-01 00:44:42,967 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:328.344752, valid_AUC:0.520675, valid_pAUC:0.503489
100%|██████████| 1803/1803 [00:26<00:00, 68.57it/s]
2020-11-01 00:45:35,263 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:249.191595, valid_AUC:0.520832, valid_pAUC:0.503423
100%|██████████| 1803/1803 [00:27<00:00, 65.89it/s]
2020-11-01 00:46:28,201 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:247.756105, valid_AUC:0.520909, valid_pAUC:0.503451
100%|██████████| 1803/1803 [00:27<00:00, 66.02it/s]
2020-11-01 00:47:21,610 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:247.378185, valid_AUC:0.520875, valid_pAUC:0.503432
100%|██████████| 1803/1803 [00:27<00:00, 65.75it/s]
2020-11-01 00:48:14,459 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:247.878977, valid_AUC:0.521012, valid_pAUC:0.503432
100%|██████████| 1803/1803 [00:27<00:00, 66.19it/s]
2020-11-01 00:49:05,940 - pytorch_modeler.p

100%|██████████| 1803/1803 [00:26<00:00, 69.17it/s]
2020-11-01 01:14:51,984 - pytorch_modeler.py - INFO - Epoch 36/100:train_loss:247.897274, valid_AUC:0.520854, valid_pAUC:0.503404
100%|██████████| 1803/1803 [00:26<00:00, 68.66it/s]
2020-11-01 01:15:42,606 - pytorch_modeler.py - INFO - Epoch 37/100:train_loss:248.454240, valid_AUC:0.520552, valid_pAUC:0.503461
100%|██████████| 1803/1803 [00:25<00:00, 69.38it/s]
2020-11-01 01:16:33,112 - pytorch_modeler.py - INFO - Epoch 38/100:train_loss:247.510315, valid_AUC:0.520957, valid_pAUC:0.503461
100%|██████████| 1803/1803 [00:26<00:00, 68.86it/s]
2020-11-01 01:17:23,775 - pytorch_modeler.py - INFO - Epoch 39/100:train_loss:247.934071, valid_AUC:0.520323, valid_pAUC:0.503423
100%|██████████| 1803/1803 [00:26<00:00, 67.97it/s]
2020-11-01 01:18:13,839 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:247.389305, valid_AUC:0.520775, valid_pAUC:0.503423
100%|██████████| 1803/1803 [00:26<00:00, 68.52it/s]
2020-11-01 01:19:04,657 - pytorch_mode

100%|██████████| 1803/1803 [00:26<00:00, 68.58it/s]
2020-11-01 01:44:45,553 - pytorch_modeler.py - INFO - Epoch 72/100:train_loss:247.838007, valid_AUC:0.520880, valid_pAUC:0.503395
100%|██████████| 1803/1803 [00:26<00:00, 67.21it/s]
2020-11-01 01:45:37,633 - pytorch_modeler.py - INFO - Epoch 73/100:train_loss:248.586701, valid_AUC:0.521206, valid_pAUC:0.503423
100%|██████████| 1803/1803 [00:26<00:00, 67.44it/s]
2020-11-01 01:46:28,445 - pytorch_modeler.py - INFO - Epoch 74/100:train_loss:248.054332, valid_AUC:0.520871, valid_pAUC:0.503423
100%|██████████| 1803/1803 [00:25<00:00, 70.29it/s]
2020-11-01 01:47:19,960 - pytorch_modeler.py - INFO - Epoch 75/100:train_loss:248.462768, valid_AUC:0.521012, valid_pAUC:0.503386
100%|██████████| 1803/1803 [00:25<00:00, 70.38it/s]
2020-11-01 01:48:11,183 - pytorch_modeler.py - INFO - Epoch 76/100:train_loss:248.094295, valid_AUC:0.520807, valid_pAUC:0.503423
100%|██████████| 1803/1803 [00:25<00:00, 69.71it/s]
2020-11-01 01:49:01,319 - pytorch_mode

...

use: cuda:0


100%|██████████| 855/855 [00:12<00:00, 69.56it/s]
2020-11-01 07:51:06,057 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:363.923349, valid_AUC:0.492874, valid_pAUC:0.504193
100%|██████████| 855/855 [00:11<00:00, 71.53it/s]
2020-11-01 07:51:37,292 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:268.187257, valid_AUC:0.489566, valid_pAUC:0.503991
100%|██████████| 855/855 [00:12<00:00, 68.07it/s]
2020-11-01 07:52:12,130 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:266.699182, valid_AUC:0.489423, valid_pAUC:0.504020
100%|██████████| 855/855 [00:11<00:00, 73.65it/s]
2020-11-01 07:52:45,368 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:265.171371, valid_AUC:0.489264, valid_pAUC:0.504106
100%|██████████| 855/855 [00:12<00:00, 69.58it/s]
2020-11-01 07:53:18,098 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:264.892757, valid_AUC:0.489209, valid_pAUC:0.504020
100%|██████████| 855/855 [00:12<00:00, 70.22it/s]
2020-11-01 07:53:51,455 - pytorch_modeler.py - INFO - E

100%|██████████| 855/855 [00:11<00:00, 74.04it/s]
2020-11-01 08:11:29,604 - pytorch_modeler.py - INFO - Epoch 37/100:train_loss:253.586475, valid_AUC:0.488725, valid_pAUC:0.503730
100%|██████████| 855/855 [00:12<00:00, 66.95it/s]
2020-11-01 08:12:05,846 - pytorch_modeler.py - INFO - Epoch 38/100:train_loss:253.490284, valid_AUC:0.488549, valid_pAUC:0.503586
100%|██████████| 855/855 [00:11<00:00, 73.99it/s]
2020-11-01 08:12:39,402 - pytorch_modeler.py - INFO - Epoch 39/100:train_loss:254.389250, valid_AUC:0.488750, valid_pAUC:0.503759
100%|██████████| 855/855 [00:12<00:00, 68.32it/s]
2020-11-01 08:13:14,635 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:253.610919, valid_AUC:0.488582, valid_pAUC:0.503702
100%|██████████| 855/855 [00:11<00:00, 72.85it/s]
2020-11-01 08:13:48,819 - pytorch_modeler.py - INFO - Epoch 41/100:train_loss:252.854171, valid_AUC:0.488544, valid_pAUC:0.503730
100%|██████████| 855/855 [00:12<00:00, 67.64it/s]
2020-11-01 08:14:22,347 - pytorch_modeler.py - INF

100%|██████████| 855/855 [00:11<00:00, 73.88it/s]
2020-11-01 08:31:43,306 - pytorch_modeler.py - INFO - Epoch 73/100:train_loss:253.512082, valid_AUC:0.488390, valid_pAUC:0.503673
100%|██████████| 855/855 [00:12<00:00, 69.08it/s]
2020-11-01 08:32:16,915 - pytorch_modeler.py - INFO - Epoch 74/100:train_loss:253.005328, valid_AUC:0.488742, valid_pAUC:0.503673
100%|██████████| 855/855 [00:11<00:00, 74.53it/s]
2020-11-01 08:32:51,004 - pytorch_modeler.py - INFO - Epoch 75/100:train_loss:253.848543, valid_AUC:0.488544, valid_pAUC:0.503702
100%|██████████| 855/855 [00:12<00:00, 69.06it/s]
2020-11-01 08:33:22,283 - pytorch_modeler.py - INFO - Epoch 76/100:train_loss:253.423405, valid_AUC:0.488538, valid_pAUC:0.503673
100%|██████████| 855/855 [00:13<00:00, 65.33it/s]
2020-11-01 08:33:58,918 - pytorch_modeler.py - INFO - Epoch 77/100:train_loss:253.995933, valid_AUC:0.488176, valid_pAUC:0.503730
100%|██████████| 855/855 [00:12<00:00, 70.94it/s]
2020-11-01 08:34:30,367 - pytorch_modeler.py - INF

elapsed time: 3380.103407621 [sec]


In [12]:
#history = run('ToyCar')
#with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
#    pickle.dump(history , file)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#history['reconstruct_img']['label']